In [382]:
import pandas as pd
import numpy as np
from uszipcode import SearchEngine

In [383]:
search = SearchEngine(simple_zipcode=True)

In [395]:
df = pd.read_csv('tj_zips.csv')
df.rename(columns={'Zip Code': 'Zip_Code', 'Population (2010 Census)': 'Pop_2010' }, inplace=True)
df = df.iloc[:, :4]
df.dropna(subset=['Zip_Code'], inplace=True)
df.drop(df[df['Zip_Code'].str.contains(',')].index, inplace=True)
df['Zip_Code'] = df.Zip_Code.astype(np.int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41318 entries, 0 to 41318
Data columns (total 4 columns):
Zip_Code    41318 non-null int64
City        41318 non-null object
State       41318 non-null object
Pop_2010    33497 non-null object
dtypes: int64(1), object(3)
memory usage: 1.6+ MB


In [410]:
## Using uszipcode library
df['Pop_2010_uszipcode'] = [search.by_zipcode(str(z)).population for z in df['Zip_Code']]

In [396]:
df.head()

,Zip_Code,City,State,Pop_2010
0,99546,Adak,AK,326
1,99571,Adak,AK,160
2,99615,Akhiok,AK,"12,899"
3,99551,Akiachak,AK,627
4,99552,Akiak,AK,346


In [397]:
## Using outside data source
zf = pd.read_csv('https://query.data.world/s/op67ehfhbfiugjxpzpr6rk3vfouzxp')
zf.columns = [col.replace('-', '_') for col in zf.columns]
common = set(df.Zip_Code) & set(zf.zip_code)
zf.set_index('zip_code', inplace=True)

In [398]:
zf.head()

,y_2016,y_2015,y_2014,y_2013,y_2012,y_2011,y_2010,aggregate
zip_code,,,,,,,,
601,17800,17982,18088,18450,18544,18533,18570,127967
602,39716,40260,40859,41302,41640,41930,41520,287227
603,51565,52408,53162,53683,54540,54475,54689,374522
606,6320,6331,6415,6591,6593,6386,6615,45251
610,27976,28328,28805,28963,29141,29111,29016,201340


In [411]:
## Using dataworld link

years = range(2010, 2017)
na = np.nan

for year in years:
    if year == 2010:
        name, yr = 'Pop_{}_dataworld'.format(year), 'y_{}'.format(year)
        df[name] = [zf.at[x, yr] if x in common else na for x in df.Zip_Code]

In [412]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41318 entries, 0 to 41318
Data columns (total 12 columns):
Zip_Code              41318 non-null int64
City                  41318 non-null object
State                 41318 non-null object
Pop_2010              33497 non-null object
Pop_2010_dataworld    32987 non-null float64
Pop_2011_dataworld    32987 non-null float64
Pop_2012_dataworld    32987 non-null float64
Pop_2013_dataworld    32987 non-null float64
Pop_2014_dataworld    32987 non-null float64
Pop_2015_dataworld    32987 non-null float64
Pop_2016_dataworld    32987 non-null float64
pop_2010_uszipcode    32987 non-null float64
dtypes: float64(8), int64(1), object(3)
memory usage: 4.1+ MB


In [366]:
df['Pop_2010_dataworld'] = [zf.at[x, 'y_2015'] if x in common else np.nan for x in df.Zip_Code]

In [378]:
df[df.pop_2010_uszipcode != df.Pop_2010_dataworld][:10]

,Zip_Code,City,State,Pop_2010,Pop_2010_dataworld,pop_2010_uszipcode
20,99509,Anchorage,AK,NaN,NaN,NaN
22,99511,Anchorage,AK,NaN,NaN,NaN
24,99514,Anchorage,AK,0,NaN,NaN
30,99520,Anchorage,AK,--,NaN,NaN
31,99521,Anchorage,AK,--,NaN,NaN
32,99522,Anchorage,AK,--,NaN,NaN
33,99523,Anchorage,AK,--,NaN,NaN
34,99524,Anchorage,AK,--,NaN,NaN
35,99529,Anchorage,AK,--,NaN,NaN
36,99530,Anchorage,AK,--,NaN,NaN


In [57]:
## Write out to csv file.
df.to_csv('tjs_zipcodes_out.csv')

In [381]:
df[df['Pop_2010'].isna()][:10]

,Zip_Code,City,State,Pop_2010,Pop_2010_dataworld,pop_2010_uszipcode
20,99509,Anchorage,AK,NaN,NaN,NaN
22,99511,Anchorage,AK,NaN,NaN,NaN
20185,65055,Mc Girk,MO,NaN,NaN,NaN
33501,38401,Columbia,TN,NaN,54469.0,54469.0
33502,38402,Columbia,TN,NaN,NaN,NaN
33503,38223,Como,TN,NaN,NaN,NaN
33504,37316,Conasauga,TN,NaN,NaN,NaN
33505,37922,Concord,TN,NaN,33180.0,33180.0
33506,37934,Concord,TN,NaN,24107.0,24107.0
33507,37933,Concord Farragut,TN,NaN,NaN,NaN
